In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import matplotlib.pyplot as plt
train_df=pd.read_csv("E:\\assignment\\ASS 11 women in data science\\train.csv")
train_df.head()
test_df=pd.read_csv("E:\\assignment\\ASS 11 women in data science\\test_QkPvNLx.csv")
big_df=train_df.append(test_df,sort=False)
big_df.head()
big_df['Long_Promotion'].unique()
big_df['Sales'].fillna((big_df['Sales'].mean()),inplace=True)
big_df['User_Traffic'].fillna((big_df['User_Traffic'].median()),inplace=True)
big_df['Competition_Metric'].fillna((big_df['Competition_Metric'].median()),inplace=True)
sxfe=['User_Traffic','Competition_Metric']
data=big_df[sxfe]
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc1= StandardScaler()
X_sc = sc_X.fit_transform(data)
cd=pd.get_dummies(big_df['Course_Domain'],drop_first=True)
ct=pd.get_dummies(big_df['Course_Type'],drop_first=True)
df = pd.DataFrame(data=X_sc)
big_df = pd.concat([big_df,cd,ct],axis=1)
big_df.drop(['ID','Day_No','Course_ID','User_Traffic','Competition_Metric','Course_Domain','Course_Type'],axis=1,inplace=True)
f=['Short_Promotion','Public_Holiday','Long_Promotion','Sales','Development','Finance & Accounting','Software Marketing','Degree','Program']
xfea=big_df[f]
df.reset_index(drop=True, inplace=True)
xfea.reset_index(drop=True, inplace=True)

big_dfx = pd.concat([df,xfea],axis=1)
df_train=big_dfx[0:512087]
df_test=big_dfx[512087:]
X=df_train.drop(['Sales'],axis=1)
y=df_train.Sales

Xtest=df_test.drop(['Sales'],axis=1)

In [2]:
import xgboost as xgb

C:\ProgramData\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\ProgramData\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [5]:
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
n_estimators =[100,200]
learning_rate = ['0.1']
random_grid = {'n_estimators': n_estimators,'learning_rate': learning_rate}
regressor=xgb.XGBRegressor()
xg_random = RandomizedSearchCV(estimator = regressor, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)
xg_random.fit(X,y)
y_pred=xg_random.predict(Xtest)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:271: UserWarning: The total space of parameters 2 is smaller than n_iter=100. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:  3.1min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  4.2min finished


In [6]:
results = np.array(y_pred)

results = pd.Series(results,name="pred")

submission = pd.concat([pd.Series(range(1,36001),name = "ImageId"),results],axis = 1)

submission.to_csv("xgboost.csv",index=False)